In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('Resources/charity_data.csv')
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
df_1 = df.drop(['EIN', 'NAME'], axis="columns")

In [3]:
#Columns to look at for binning- Application Type, 17 values & Classification, 71 values. Ask_Amt should have many different values so don't bin that one. 
#Application type
app_value = df_1["APPLICATION_TYPE"].value_counts()

other_app_type = list(app_value[app_value<150].index)


#replace "other_app_types" in dataframe
for app in other_app_type: 
    df_1['APPLICATION_TYPE'] = df_1["APPLICATION_TYPE"].replace(app,"Other")
df_1['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [4]:
#Create "Other" category for Classification
class_value = df_1["CLASSIFICATION"].value_counts()

#use 150 as above for consistency
other_class_type = list(class_value[class_value<150].index)


#replace "other_app_types" in dataframe
for cls in other_class_type: 
    df_1['CLASSIFICATION'] = df_1["CLASSIFICATION"].replace(cls,"Other")
df_1['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [5]:
#Use pd.dummies to encode categorical variables
df_dummies = pd.get_dummies(df_1, dtype=float)


In [6]:
#import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [7]:
#Set the variable and remove it from the dataframe
y = df_dummies.IS_SUCCESSFUL.values
X = df_dummies.drop(columns="IS_SUCCESSFUL").values


In [8]:
#Split train/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [9]:
#Preprocess the dataframe for a neural network
scaler = StandardScaler()

#Fit the StandardScaler
X_scaler = scaler.fit(X)

#scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
#Increase units to the first and second dense layer from 100 to 200 units

#Define the model (nn=neural network)
nn_model = tf.keras.models.Sequential()

#add Dense layers
nn_model.add(tf.keras.layers.Dense(units = 200, activation="relu", input_dim=47))
nn_model.add(tf.keras.layers.Dense(units = 200, activation="relu"))


#add output layer
nn_model.add(tf.keras.layers.Dense(units = 1, activation="sigmoid"))

#compile the model
nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

#Train model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

#Evaluate model
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5683 - accuracy: 0.7215
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5534 - accuracy: 0.7332
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5497 - accuracy: 0.7338
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7342
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5473 - accuracy: 0.7348
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7343
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5455 - accuracy: 0.7357
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5452 - accuracy: 0.7355
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5447 - accuracy: 0.7360
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5440 - accuracy: 0.7364